In [10]:
import sys
import serial

# ToDo: Erweiterung verschiedener Farbräume
# ToDo: Entkopplung Matrix und Schnittstelle
#       Aktuell wird ein Schnittstellen-Objekt vom Objekt der Klasse RGBBasis aufgerufen. So ist es z.B. nicht möglich
#       an das gleiche Schnittstellen-Objekt verschiedene Objekte der RGBBasis zu schicken.
# ToDo: Grösse der Matrix frei wählbar, nicht auf 8 x 8 fixiert
# ToDo: Ausgabe png-Datei ergänzen
# ToDo: Datenkapselung mit get und set umsetzen


class RGBBasis:
    """
    Basisklasse RGB-Matrix
    """


ModuleNotFoundError: No module named 'serial'

## RGBBasis.\_\_init__()
- Zu Parameter `port`:
 - Ist dieses Parameter nötig so mein Objekt funktionnal ist? **JA**
 - Kann ich ein Vorgabewert haben der **immer** funktionniert? **NEIN**, die COM Nummer werden vom Betriebsystem vorgeben und sind nicht vorsehbar
 - Kann meine Klasse über anderen Wege das COM Nummer herausfinden? **NEIN**, im schliemsten Fall kann ich mehrere LED-Array am Komputer angehängt haben, ohne möglichkeit einer auszuwählen.
 - **Dieses Parameter-Wert muss vom Benutzer gegeben werden, er kann kein Vorgabewert haben.**


- Bei `default_color=(0, 0, 0)` ist der Vorgabewert ein Tuple (nicht änderbare Liste), in die Dokumentation steht aber `list, [R, G, B]` (änderbare Liste), was sich wiederspricht


- `self.init_rgb_matrix()` muss `self.rgb_matrix` selber ändern, nicht eine "leere" Matrix zurückgeben. Oder dann in `self.get_default_rgb_matrix()` umbennant werden. Oder noch besser und flexibler und wiederverwendbarer `self.get_filled_matrix(color)` sein.


- `self.rgb_bytestream = self.init_rgb_bytestream()`ist unnötig. In **Pyton** muss kein Speicherplatz reserviert werden. Der Bytestream kann *on the fly* generiert werden. Er wird auch nicht von mehrere Funktionnen gebraucht.

In [11]:
    def __init__(self, port='', default_color=(0, 0, 0)):
        """
        RGBBasis Objekt Konstruktor
        :param port: string, COM-Port -> siehe Geräte-Manager
        :param default_color: list, [R, G, B] -> [0..255, 0..255, 0..255]
        https://de.wikipedia.org/wiki/RGB-Farbraum

        Attribut: "rgb_matrix[zeile][spalte]" wird mit default_color initialisiert (zeile = 0..7, spalte = 0..7)
        """
        self.port = port
        self.default_color = default_color
        self.interface = UART2FPGA(enable=True, port=self.port)  # Windows 'COM.. => siehe Geräte Manager'
        self.rgb_matrix = self.init_rgb_matrix()
        self.rgb_bytestream = self.init_rgb_bytestream()

## RGBBasis.init_rgb_matrix()
- Der Name passt nicht mit die Funktion. Es währe besser mit `get_default_matrix()`


- `range(Startwert, Endwert, Schritt)` hat *Startwert=0* und *Schritt=1* als Vorgabewerten. `range(8)`reicht.


- `led.extend([rgb])` macht folgendes:
  - `[rgb]` erzuegt eine neue Liste mit die Liste *rgb* als Inhalt (*\[ [ 10, 20, 30) ] ] zum Beispiel*)
  - `led.extend(...)̀  fügt jedes Element von der oben erzeugte Liste (nur ein Element gross...) am Ende der *led*-Liste ein
  - **Um nur ein Element am Ende eine Liste zuzufügen gibt es die Funktion `liste.append(element)`**


- Um Liste zu initialisieren gibt es *list comprehension* die viel kompakter ist (und mehr *Pythonish*). Es muss aber aufgepasst werden, den bei der  `*`-Operator werden die *Elemente*, wenn nicht von eine einfachen Typ (wie *int* z.B.), nicht kopiert, nur referenziert (es ist immer den gleichen Element).


- *Breite* und *Höhe* Können als parameter gegeben werden.

In [12]:
    def init_rgb_matrix(self):
        """
        Initialisierung der 8 x 8 x 3 Matrix für die vereinfachte Farbzuweisung.
        Dimension 8 x 8 ist fix definiert!
        """
        rgb_matrix = []
        for i in range(0, 8, 1):
            led = []
            for j in range(0, 8, 1):
                rgb = self.default_color
                led.extend([rgb])
            rgb_matrix.extend([led])
        return rgb_matrix

## RGBBasis.init_rgb_matrix() neu geschrieben
- `list[:]` macht ein *slice* von die Liste *list* von Start bis Ende (Vorgabenwerte). Es erzeugt eine einfache Kopie von der Liste. **ACHTUNG:** die Unterelemente der Liste werden nicht kopiert wenn sie änderbar (*mutable*, wie Listen z.B.) sind. Sie werden nur neu referenziert.


- `deepcopy()` macht die gleiche Kopie, aber bis zum tiefstem Unterelement.


- Die zwei Codezeilen könnten in eine Zeile geschrieben sein, es ist jedoch lesbarar so.


- Um kompatible zu bleiben, wir trodzdem die rgb_matrix zurückgegeben

In [13]:
    from copy import deepcopy
    def init_rgb_matrix(self, height=8, width=8):
        """
        Initialisierung der Matrix mit alle Zellen=self.default_color.
        """
        row = [self.default_color[:] for _ in range(width)]
        rgb_matrix = [deepcopy(row) for _ in range(height)]
        
        self.rgb_matrix = rgb_matrix

        return rgb_matrix

## RGBBasis.init_rgb_bytestream()
- Es ist nicht nötig in Python Speicher zu "reservieren". Diese Liste kann "on the fly" generiert werden.


- Eine schönere Variante wäre `return [0]*(8*8*3)` gewesen

In [14]:
    @staticmethod
    def init_rgb_bytestream():
        """
        Initialisierung des 8 x 8 x 3 Bytestreams (Snake-Line), welcher für die Übertragung an die RGB-Matrix (HW)
        Dimension 8 x 8 ist fix definiert!
        https://www.led-genial.de/DIGI-DOT-Panel-8x8-HD-mit-64-x-Digital-LEDs
        """
        rgb_bytestream = []
        for i in range(0, (8 * 8 * 3), 1):
            rgb_bytestream.extend([0])
        return rgb_bytestream  

## RGBBasis.rgb_matrix_to_rgb_bytestream()
- Wieso muss hier ein Bytestream generiert werden? Wenn man die sende Funktion (`UART2FPGA.write(self, bytestream`) anschaut, sieht man das sie wieder Daten herausfinden muss (alles wieder in Zeile und Spalten trennen).


- Die Daten werden in 'Snake' umgewandelt (eine Zeile von Links nah Rechts, die nächste von Rechts nach Links usw.), und die Reihenfolge von Rot, Grün und Blau in Grün, Rot, Blau umgewandelt. Das kann auch mit erhalt der Struktur gemacht werden.


- Bei der Bearbeitung wird ein gleich ausehendes Code 6 Mal wiederholt, mit Komplexe Index verrechnung!


- Die Kommentar von die Frben sind Falsch !


- Das Slice `list[::-1]` das eine **NEUE** umgekehrte Liste generiert wird 3 mal auf die **gleiche Liste** ausgeführt.


- `(i % 2)` gibt 0 oder 1 zurück. 0 wird als Fals gesehen, alle andere Zahlen als True. `if not (i % 2):` ist genügend.


- Die RGB Matrix hat keine Ahnung (und muss nie haben) wie und in welche Reihenfolge und Form die Daten zum LED-Modul gesendet werden. Das soll ein Treiber überlassen werden.

In [15]:
    def rgb_matrix_to_rgb_bytestream(self):
        """
        Transformation der Matrix-Darstellung in die entsprechende Bytestream-Darstellung (Datenstrom).
        """
        for i in range(0, 8, 1):
            for j in range(0, 8, 1):
                if (i % 2) == 0:
                    self.rgb_bytestream[(i * 8 * 3) + (j * 3) + 0] = self.rgb_matrix[i][j][1]  # ROT
                    self.rgb_bytestream[(i * 8 * 3) + (j * 3) + 1] = self.rgb_matrix[i][j][0]  # GRUEN
                    self.rgb_bytestream[(i * 8 * 3) + (j * 3) + 2] = self.rgb_matrix[i][j][2]  # BLAU
                else:
                    self.rgb_bytestream[(i * 8 * 3) + (j * 3) + 0] = ((self.rgb_matrix[i])[::-1])[j][1]  # ROT
                    self.rgb_bytestream[(i * 8 * 3) + (j * 3) + 1] = ((self.rgb_matrix[i])[::-1])[j][0]  # GRUEN
                    self.rgb_bytestream[(i * 8 * 3) + (j * 3) + 2] = ((self.rgb_matrix[i])[::-1])[j][2]  # BLAU

   

## RGBBasis.rgb_matrix_to_rgb_bytestream() neu geschrieben
- `zip(['a', 'b', 'c'], [1, 2, 3])` generiert eine Mischung aus jeder Elemente von Liste A mit der gleichplatziertes Elemente von Liste B => ('a', 1), ('b', 2), ('c', 3), bis eine Liste leer ist


- `itertool.cycle(list)` wiedergibt jeder Element von *list* und fängt wieder am Anfang wenn alle Elemente durh sind.

In [16]:
    def rgb_matrix_to_rgb_bytestream(self):
        """
        Transformation der Matrix-Darstellung in die entsprechende Bytestream-Darstellung (Datenstrom).
        """
        self.rgb_bytestream = [] # Leere Matrix-Liste herstellen
        for zaehler, zeile in zip(itertools.cycle(Range(8), Range(7, -1, -1)), self.rgb_matrix):
            byte_zeile = [] # Neue Zeile-Liste herstellen
            
            # Und am Ende der Marix-Liste zufügen
            # Dass kKann auch nach die For Schlaufe gemacht werden
            rgb_bytestream.append(byte_zeile) 
            
            # Und die Zeile mit die LED-Farben füllen
            # Bei der erste Schlaufe ist zaehler=Range(8) -> [0, 1, 2, 3, 4, 5, 6, 7]
            # Bei der nächte Schlaufe ist zaehler=Range(7, -1, -1) -> [7, 6, 5, 4, 3, 2, 1, 0]
            for spalte in zaehler:
                r, g, b = zeile[spalte] # das ist "unboxing": die Werte der Liste werden zu die Einzelvariable vergeben
                byte_zeile.append((g, r, b)) # Und eine neu Tuple(oder Liste, geht auch) zu die Zeile zufügen


Da habe ich nichts zu sagen :-)

In [17]:
    def open_interface(self):
        """
        Öffnen der entsprechenden Schnittstelle, z.B. UART
        """
        self.interface.open()

    def close_interface(self):
        """
        Schliessen der entsprechenden Schnittstelle, z.B. UART
        """
        self.interface.close()

## RGBBasis.write2rgb()
- `self.rgb_bytestream` wird nur hier benutzt. `self.rgb_matrix_to_rgb_bytestream()` könnte es als Rückgabewert schreiben, ohne über eine Instanz-Variable zu gehen

In [18]:
    def write2rgb(self):
        """
        Entsprechende Matrix auf der RGB-Matrix darstellen.
        1. Transformation
        2. Werte auf Schnittstelle schreiben
        """
        self.rgb_matrix_to_rgb_bytestream()
        self.interface.write(self.rgb_bytestream)

## RGBBasis.\_\_str__ und RGBBasis.\_\_repr__
- `__str__()` gibt eine "human readable" Darstellung der Objekt zurück


- `__repr__()` gibt eine "offizielle" Darstellung zurück. Normalerweise kann diese String als Parameter an `eval()` gegeben werden und diese bearbeitet sie als währe es ein Python Befehl.


- Im Fall des RGBBasis Klasse, könnten beide die gleiche Zeichenkette zurückgeben, ohne das es Schwirigkeit gibt für der Benutzer deise zu verstehen.


- in `default_color={:}` macht der **:** kein Sinn: er soll ein formatierer einführen, da ist aber keiner. 

In [19]:
    def __str__(self):
        """
        Klartextausgabe der Klasseneigenschaften
        """
        return 'RGBBasis(Port={}'.format(self.port) + ', default_color={:}'.format(self.default_color) + ')'

    def __repr__(self):
        return 'RGBBasis(port=\'{}\''.format(self.port) + ', default_color={:}'.format(self.default_color) + ')'

## UART2FPGA.\_\_init__()
- wofür wird self.enable benutzt? Es wird nur getestet, nie geschrieben. Was passiert wenn folgendes Code ausgeführt wird ?
 - `uart = UART2FPGA(enable=False)
uart.open()
uart.enable = True
uart.write(b'Some bytes')
uart.close()`
 - Die Schnitstelle wird nicht geöffnet, die aufrufendes Funktion weiss es aber nicht (Feedback durch `print()`!!!!! **Ich lese nur die Fehlerausgabe (Exceptions) meines Program. Ich schaue sonnst auf der LED-Matrix, ob was passiert.**)
 - `uart.write()` versucht etwas auf die Schnitstelle zu shreiben (`enable == True`) ohne zu prüfen ob es je geöffnet worden ist, und beendet mein Programm.


- in `port='COM43` woher kommt das COM Port Nummer 43? Wenn es ein Beispeil sein soll, muss es in die Kommentare erscheinen. Wenn der Parameter obligatorisch (siehe Kommentar zu `RGBBasis.__init__()`)


- `self.instanz`. Ja, aber *Instanz* von **was**? `self.serial_interface` z.B.

In [ ]:
class UART2FPGA:
    """
    Klasse für die Beschreibung der Schnittstelle mittels UART
    https://de.wikipedia.org/wiki/Universal_Asynchronous_Receiver_Transmitter
    """
    def __init__(self, enable=True, port='COM43'):
        """
        UART2FPGA Objekt Konstruktor
        :param enable: boolean, Schnittstelle aktivieren?
        :param port: string, COM-Port -> siehe Geräte-Manager
        """
        self.enable = enable
        self.port = port
        self.instanz = None

## UART2FPGA.open()
- Nur als Spass, wenn `self.port == None` (wird in `__init__` nicht verweigert), wird die Schnitstelle nicht geöffnet, und gibt eine Fehler heraus bei `write()`.
>The port is immediately opened on object creation, when a port is given. It is not opened when port is `None` and a successive call to `open()` is required.

- Die Fehler mit `try` und `except` zu behandeln ist super. Was kann aber dieses Objekt schlaues machen wenn eine vorkommt? *SICHER NICHT DIESE AUSDRUCKEN UND **DAS PROGRAMM BEEDEN***(`sys.exit()`).
 - Ein Program hat normalerweise 2 Ausgabe-Ströme:
   - stdout : alle `print()` ohne weitere Info werden darauf geschrieben. Für "Benutzer-Austauch" gemeint
   - errout : **alle** Fehlermeldungen (aussert infrmativ) sollten darauf geschrieben werden. Der Benutzer sieht sie nicht unbedingt (oder sogar in eine andere Frbe, wie Rot z.B), sie können aber von eine Aufrufendes Programm gelesen werden um zu wisen was passiert ist.
 - Wenn die Fehler-Meldungen durch `print()` auf *stdout* umgeleitet werden, wirde die Informtion das es sich um eiene Fehler handelt **verloren**
 - `sys.exit()` nimt hat als Parameter das Ergebnis des Program.
   - **> 0** -> Alles gut gelaufen, mit folgende Info (Parameter-Wert)
   - **= 0** -> Alles gut gelaufen, nicht weiteres zu sagen
   - **< 0** -> Es ist eine Fehler aufegtreten, mit folgendes Code (Parameter-Wert)
 - Ohne Parameter gibt es 0 heraus, so **alles gut gelaufen**!!!! Gerade das **Gegenteil** von was geschehen ist.
 - Nehmen wir an, ich binde dieses Objekt in eine Grafische Oberfläche. Wenn eine Kommunikationsfehler auftritt, stürtzt mein Programm einfach ab, ohne Informtion am benutzer oder mögliche weitere Massnahmen. Was eine Frechheit!!!!
 
- Es gibt zwei Möglichkeiten:
  - gar nichts machen, die *Exception* wird an die Aufrufende Funktionen weitergegeben, bis eine diese bearbeitet, oder der Program abstürtzt (aber auf eine *sauberes* Weg)
  - Eine neue *persönliche* Exception generieren (z.B. "Fehler bei Kommunikation mit die Schnitstelle") und diese senden.

In [ ]:
    def open(self):
        """
        Oeffne UART mit der COM - Bezeichnung
        """
        if self.enable:
            try:
                self.instanz = serial.Serial(self.port, 921600, timeout=0, parity=serial.PARITY_NONE)
            except Exception as e:
                print('uart_open', e)
                sys.exit()
        else:
            print('UART_ENABLE == False')

## UART2FPGA.write()
- wenn `self.enable` nicht gesetzt ist oder die Länge des *bytestream* nicht stimmt, passiert einfach nichts, ohne das die aufrufende Funktion etwas davon mitbekommt.

- es wird 3 mal `uart_string += '{:02X}'.format(bytestream[(i * 8 * 3) + (j * 3) + ...])` benutzt. Eine dritte Schlaufe ist hier nötig

- Variabelname wie *i* und *j* müssen vermeidet werden, in besonderen wenn sie was *physicalisches* einthalten. *zeile* und *spalte* hätten hier mehr bedeutung und vereinfachen das lesen.

- `range(0, 8, 1)` <=> `range(8)`

- *i* wird **immer** durch (8\*3) multipliziert. `range(8*8*3, 8*3)` vermeidet das mehrmals multiplizieren. `range(8*3, 3)` fur *j* das selbe

- `string` addieren fürt zu erzeugen eine neue Kopie der string jedes mal. Efizienter ist alle einzelne Teile in eine Liste zuzufügen und am Schluss `uart_string = ''.join(liste)` zu benutzen. Es wird eine Zeichenkette generiert aus die zusammenfügung jeder Element der Liste getrennt durch die aufrufende Zeichenkette (`''` -> ohne Trennung, `';'.join(['1', '2', '3'])` -> `'1;2;3'`)


In [20]:
    def write(self, bytestream):
        """
        Schreiben auf den UART Kanal
        """
        if self.enable:
            try:
                if len(bytestream) == 8 * 8 * 3:
                    for i in range(0, 8, 1):
                        uart_string = '<{:02x}'.format(i + 1)  # https://pyformat.info/
                        for j in range(0, 8, 1):
                            uart_string += '{:02X}'.format(bytestream[(i * 8 * 3) + (j * 3) + 0])
                            uart_string += '{:02X}'.format(bytestream[(i * 8 * 3) + (j * 3) + 1])
                            uart_string += '{:02X}'.format(bytestream[(i * 8 * 3) + (j * 3) + 2])
                        uart_string += '>'
                        self.instanz.write(uart_string.encode())  # write a string
            except Exception as e:
                print('uart_write', e)
                sys.exit()

    def close(self):
        """
        Schliessen des UART Kanals.
        """
        if self.enable:
            try:
                self.instanz.close()
            except Exception as e:
                print('uart_close', e)
                sys.exit()

    def __str__(self):
        """
        Klartextausgabe der Klasseneigenschaften
        """
        return 'UART2FPGA(enable={}'.format(self.enable) + ', port={}'.format(self.port) + ')'

    def __repr__(self):
        return 'UART2FPGA(enable={}'.format(self.enable) + ', port=\'{}\''.format(self.port) + ')'